# Bayesian Treatment and Response

In this notebook we are going to outline a method for estimating treatment response models using some Bayesian methods. In this notebook, we will discuss the discrete treatment (i.e., there is a treatment group and a control group like belonging to a union or not), with continuous response (the response we are interested in is a continuous variable, like wages). 

Why would someone resort to Bayesian methods to estimate this model, as opposed to the classical alternatives such as Instrumental-Variables probit, or even regular linear IV? A couple of reasons:

1. Bayesian methods usually have a lot of structure in that the involve a lot of distributional assumptions. Hence, they can be pretty good in small samples. 
2. Bayesian methods allow one to say something about things like correlation across outcomes that are typically not observable.
3. Bayesian methods also generate some things like individual-level treatment effects just as a by-product of estimation.


# The basic model and theory

We consider a case in which there is a treatment and control group and some set of control variables. In fact, we might think about this more generally as an endogenous switching regression, where agents select into different groups in part based on things we observe about them and maybe on some things we do not. 

We take treatment to be a dichotomous variable $z = \{0,1\}$, and we observe $z=1$ if some underlying latent variable $z^*$, is greater than zero, and $z=0$ otherwise. 

As a practical matter, we think of the selection-into-treatment equation as a Probit model, where:
$$
z^* = \eta W + \epsilon_z
$$
And 
$$ \begin{array}{ccc}
z = 1 & \textrm{if} & z^* = \eta W + e_z > 0 \\
z = 0 & \textrm{otherwise} &
\end{array}
$$

Outcome equations differ based on whether or not the individual is treated. If $z=1$, the following equation explains the outcome $y$:
$$
y_1 = X\beta_1 + e_1
$$
and if $z=0$,
$$
y_0 = X\beta_0 + e_0
$$

## Point of interest

In observational data, we never actually observe both outcomes for a given individual. I.e., we don't see what happens to a patient if he or she both takes and does not take a medication or we don't see what happens if someone both does and does not join a union. But a Bayesian method allows us to consider this as well - the idea is to treat the unobserved outcome as yet another latent variable that is estimated/simulated along with everything else. 

The variance matrix for $y_1,y_0$, and $z$ is:

$$
\Sigma = \left[
\begin{array}{ccc}
\sigma_1^2  & \sigma_{10} & \sigma_{1z} \\
                                   & \sigma_0^2 & \sigma_{0z} \\
                                   &      &  1 
\end{array}
\right]
$$
We have to also assume that $\sigma_z=1$ because of the indeterminacy of the scale parameter in a probit model. 






## Methods of estimation

If we wanted to write the full likelihood of the data, it might consist of three parts: a probit style likelihood describing treatment or not, and then a likelihood for the outcome conditional on treatment. This is tricky in the current setting because we have a trivariate normal distribution, and what can be thought of as two latent variables - the threshold variable for treatment, and the value of whatever is unobserved. This is tough (but not impossible) - Heckman and coauthors have some papers describing methods for dealing with these problems. But my approach will be to think directly about things in terms of a Gibbs sampling approach. 

### Gibbs sampling

To design a Gibbs sampler, one just has to think about the conditional distribution of one thing when everything else is given. Heuristically, my idea is to proceed as follows:

1. Given $\beta,\eta,\Sigma, X,y_0$, draw values for $y_1$. That is, draw a treated outcome for those who were not treated (z = 0). 
2. Given $\beta,\eta,\Sigma, X,y_1$, draw values for $y_0$. That is, draw a non-treated outcome for those who were treated (z=1).
3. Given $\eta,\Sigma, W,y_0,y_1,z$, draw values for $z^*$, the unobserved latent variable describing selection into treatment.
4. Given $\Sigma, X,y_0,y_1,z$ draw values for $\beta_0$.
5. Given $\Sigma,X,y_0,y_1,z$ draw values for $\beta_1$.
6. Draw values for the 5 terms in $\Sigma$.

Almost everything here just requires a normal distribution. Number 6 is a little more complicated, but we can use a Markov-chain Monte carlo Metropolis-within-Gibbs step for that. We need one formula, which is that of the conditional normal. If we want to know the distribution of some subset $y_1$ given $y_2$, when mean vectors are $\mu_1, \mu_2$ and the variance matrix is $\Sigma$, we use:

\begin{equation*}
\Sigma = \left[ \begin{array}{c|c}
\Sigma_{11} & \Sigma_{12} \\
\hline
\Sigma_{21} & \Sigma_{22}
\end{array}\right]
\end{equation*}

The conditional mean of $y_1$ given $y_2$ is:
\begin{equation*}
E[y_{1|2}] = \mu_1 + \Sigma_{12}\Sigma_{22}^{-1}(y_2 - \mu_2) 
\end{equation*}
and the conditional variance of $y_1$ given $y_2$ is:
\begin{equation*}
V[y_{1|2}] = \Sigma_{11} - \Sigma_{12}\Sigma_{22}^{-1}\Sigma_{21} 
\end{equation*}



**Step 1:**

Use $\mu_1=X\beta_1$, $\mu_2=X\beta_0, W\eta$, $y_2=y_0, z$, $\Sigma_{12} = \Sigma_{21}' =\left[\sigma_{10},\sigma_{1z}\right]$,
and
\begin{equation*}
\Sigma_{22}=\left[\begin{array}{cc}\sigma_0^2 & \sigma_{0z} \\ \sigma_{0z} & 1 \end{array} \right]
\end{equation*}





**Step 2:**

Use $\m1_0=X\beta_0$, $\mu_2=[X\beta_1,W\eta]$, $y_2=[y_1, z]$, $\Sigma_{12} = \Sigma_{21}' =\left[\sigma_{10},\sigma_{0z}\right]$,
and
\begin{equation*}
\Sigma_{22}=\left[\begin{array}{cc}\sigma_1^2 & \sigma_{1z} \\ \sigma_{1z} & 1 \end{array} \right]
\end{equation*}


**Step 3: **

Use $\mu_1 = W\eta$, $\mu_2=[X\beta_1,X\beta_0]$, $y_2=[y_1, y_0]$, $\Sigma_{12} = \Sigma_{21}' =\left[\sigma_{1z},\sigma_{0z}\right]$,
and
\begin{equation*}
\Sigma_{22}=\left[\begin{array}{cc}\sigma_1^2 & \sigma_{10} \\ \sigma_{10} & \sigma_0^2 \end{array} \right]
\end{equation*}

But note that draws must be from a truncated random normal variable, where if $z=1$, data is left-truncated at 0, and if $z=0$ data is right-truncated at 0.

** Step 4/5/6: **

We now have all the data that we need and all we have to do is transform the dependent variables so that they are purged of cross equation correlation. That is, we remove cross-variable correlation from $y_1,y_0,$ and $z$ and then observe that:
\begin{equation*}
\beta_1 \sim N\left[ (X'X)^{-1}X'\hat{y}_1, \sigma_{1}^2(X'X)^{-1} \right]
\end{equation*}
Similarly:
\begin{equation*}
\beta_0 \sim N\left[ (X'X)^{-1}X'\hat{y}_0, \sigma_{0}^2(X'X)^{-1} \right]
\end{equation*}
and finally
\begin{equation*}
\eta \sim N\left[ (W'W)^{-1}W'\hat{z}, (W'W)^{-1}\right]
\end{equation*}





As written, the likelihood above assumes that everything is known. Of course, many things in the problem are not, such as the unobserved outcome and also the actual value of the latent variable $z$. But, in a Bayesian analysis, we just draw these variables along with everything else. Let's consider another `Stata` implementation of this model, where we draw unobserved outcomes along with everything else. 

## Example application - an extended walkthrough

As usual, we will work with a stock Stata example and use the `ipystata` interface. Here, we use the classic union data set and try to get a feel for the impact of union membership on wages. So, in this case, we have a dichotomous treatment variable with one instrument (in the south). 

Initially, we are going to take it very slow and worry about how initial draws are produced, and think about how each step works. At the end of this, we are going to write a program to a `.do` file that will speed things up a little bit. 

In [1]:
import ipystata

In [3]:
%%stata
clear all 
use http://www.stata-press.com/data/r13/union3
set more off
keep if union != . 
keep if tenure != .
describe 
set seed 5150


(National Longitudinal Survey.  Young Women 14-26 years of age in 1968)
(449 observations deleted)
(34 observations deleted)

Contains data from http://www.stata-press.com/data/r13/union3.dta
  obs:         1,210                          National Longitudinal Survey.  Young Women 14-26 years of age in 1968
 vars:            24                          11 Mar 2013 09:47
 size:        55,660                          
---------------------------------------------------------------------------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
---------------------------------------------------------------------------------------------------------------------------------------------------
idcode          int     %8.0g                 NLS ID
year            byte    %8.0g                 interview year
birth_yr        byte    %8.0g                 birth year
age        

Now, let's just fit some preliminary models and get a feel for the problem, to get our bearings and get some ideas as to how large parameters are likely to be:

## Simple regression and IV regressions

The first thing that we want to do is just run some simple regressions, and a probit for selection into "treatment" which is here belonging to a union. In fact, we will take these as the initial values for our Bayesian estimation procedure. So:

In [5]:
%%stata
reg wage age grade smsa black tenure if union == 1
mat binit1 = e(b)

reg wage age grade smsa black tenure if union == 0
mat binit0 = e(b) 

probit union south black tenure
mat einit = e(b)



      Source |       SS           df       MS      Number of obs   =       253
-------------+----------------------------------   F(5, 247)       =     18.89
       Model |  419.610641         5  83.9221281   Prob > F        =    0.0000
    Residual |  1097.48439       247  4.44325665   R-squared       =    0.2766
-------------+----------------------------------   Adj R-squared   =    0.2619
       Total |  1517.09503       252  6.02021838   Root MSE        =    2.1079

------------------------------------------------------------------------------
        wage |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         age |   .1562246   .0492065     3.17   0.002     .0593067    .2531425
       grade |   .3897226    .062765     6.21   0.000     .2660997    .5133456
        smsa |    1.10633   .3384687     3.27   0.001     .4396772    1.772983
       black |  -.9184192   .2896539    -3.17   0.

Now, we pull all the variables that we need into `Mata`, and some constants to get the full-blown vectors we need, and at the same time, we pull in the initial values from the regressions to be used. We also set logs of variances at zero, and set all covariance terms at 0 too.

In [9]:
%%stata --mata
    st_view(y=., ., "wage")
    st_view(tr=., ., "union")
    st_view(X=., ., "age grade smsa black tenure")
    st_view(W=., ., "south black tenure")
    X = X, J(rows(y), 1, 1)
    W = W, J(rows(y), 1, 1)

    b1 = st_matrix("binit1")
    b0 = st_matrix("binit0")
    e = st_matrix("einit")
    nb = cols(b1)
    ne = cols(e)

    lnsd1 = 0
    lnsd0 = 0
    v10   = 0
    v1t   = 0
    v0t   = 0



Mata output:

:     st_view(y=., ., "wage")

:     st_view(tr=., ., "union")

:     st_view(X=., ., "age grade smsa black tenure")

:     st_view(W=., ., "south black tenure")

:     X = X, J(rows(y), 1, 1)

:     W = W, J(rows(y), 1, 1)

:     b0 = st_matrix("binit0")

:     e = st_matrix("einit")

:     nb = cols(b1)

:     ne = cols(e)

:     lnsd0 = 0

:     v10   = 0

:     v1t   = 0

:     v0t   = 0


One further thing we need to do is make a more stable inverse-normal function. This is because if a normal probability winds up being either one or zero, the stock invnormal function will return a missing. We don't want this to stop estimation in its tracks! 

We also create a function that takes error terms $y-\mu$ and a variance matrix $\Sigma$, and calculates the joint likelihood of all the data based on these things. It uses a couple of tricks to economize on notation and avoil looping

In [15]:
%%stata --mata
    real matrix invnormstab(X) {
        XHat = editvalue(X, 0, 1e-323)
        XHat = editvalue(XHat, 1, 1e-16 )
        return(XHat)
    }
    real scalar ln_L(real matrix errs, real matrix Sigma) {
        part1 = -cols(errs)*rows(errs)/2*ln(2*pi())
        part2 = -1/2*colsum(rowsum( (errs*invsym(Sigma):*errs)))
        part3 = -rows(errs)/2*ln(det(Sigma))
        return(part1 + part2 + part3)
    }

Mata output:

:     real matrix invnormstab(X) {
>         XHat = editvalue(XHat, 1, 1e-16 )
>     }

:     real scalar ln_L(real matrix errs, real matrix Sigma) {
>         part2 = -1/2*colsum(rowsum( (errs*invsym(Sigma):*errs)))
>         return(part1 + part2 + part3)

---------------------------------------------------------------------------------------------------------------------------------------------------
command end is unrecognized
r(199);


As we have initial values for all parameters, we now need to fill in some values for the latent $z$, and also for "missing" $y_1$ and $y_0$. The $y$'s are pretty simple to deal with. The idea is to fill in $y_0$ with a drawn value whenever it is missing, doing the same for $y_1$. Here is the code:

In [16]:
%%stata --mata
    y0Hat = X*b0' + rnormal(rows(y),1,0,1)*exp(lnsd0)
    y1Hat = X*b1' + rnormal(rows(y),1,0,1)*exp(lnsd1)
    y1 = tr:*y + (1 :- tr):*y1Hat
    y0 = (1 :- tr):*y + tr:*y0Hat

Mata output:

:     y0Hat = X*b0' + rnormal(rows(y),1,0,1)*exp(lnsd0)

:     y1Hat = X*b1' + rnormal(rows(y),1,0,1)*exp(lnsd1)

:     y1 = tr:*y + (1 :- tr):*y1Hat

:     y0 = (1 :- tr):*y + tr:*y0Hat


The case for $z$ is a little more complicated, because it is latent and we have to draw a truncated random normal variable. So:

In [17]:
%%stata --mata
    muz = W*e'
    et  = invnormstab( normal(-muz) + (1 :- normal(-muz)):*runiform(rows(muz),1) )
    ent = invnormstab( normal(-muz):*runiform(rows(muz),1) )
    z = muz + et:*tr + ent:*(1 :- tr)

Mata output:

:     muz = W*e'

:     et  = invnormstab( normal(-muz) + (1 :- normal(-muz)):*runiform(rows(muz),1) )

:     ent = invnormstab( normal(-muz):*runiform(rows(muz),1) )

:     z = muz + et:*tr + ent:*(1 :- tr)


One last thing that makes everything a bit easier is if we compute deviations from means and keep track of these as we go along. We don't need to update these except for when draws or parameters change. Accordingly:

In [18]:
%%stata --mata
    m0 = X*b0'
    m1 = X*b1'
    mt = W*e'
    ey1 = (y1 - m1)
    ey0 = (y0 - m0)
    et  = (z - mt)

Mata output:

:     m0 = X*b0'

:     m1 = X*b1'

:     mt = W*e'

:     ey1 = (y1 - m1)

:     ey0 = (y0 - m0)

:     et  = (z - mt)


Now, we make some placeholders for parameters, also put some prior distributions into action. We also calculate the symmetric inverse of $X'X$ and $W'W$, as these things never change so there is no great reason to keep calculating them as we run through the iterations!

In [12]:
%%stata --mata
    b1Hold        = b1
    b0Hold        = b0
    eHold         = e
    sy1Hold       = lnsd1
    sy0Hold       = lnsd0
    v10Hold       = v10
    v1tHold       = v1t
    v0tHold       = v0t

    draws = 100000
    
    XX = invsym(X'X)
    WW = invsym(W'W)

Mata output:

:     b1Hold        = b1

:     b0Hold        = b0

:     eHold         = e

:     sy1Hold       = lnsd1

:     sy0Hold       = lnsd0

:     v10Hold       = v10

:     v1tHold       = v1t

:     v0tHold       = v0t

:     
:     XX = invsym(X'X)

:     WW = invsym(W'W)


## A typical draw of a latent variable

While draws typically follow what we did above, it is helpful to just see how a typical draw and update goes for our $y_1$ variable. In this case, we have to compute the sub-matrices of $\Sigma$, calculate conditional means, and then draw. Finally, we will update the error vector. The whole thing looks a little something like this:


In [19]:
%%stata --mata

    Sig12 = (v10, v1t)
    Sig22 = exp(lnsd0)^2, v0t \ v0t, 1
    Sig22m1 = invsym(Sig22)

    CM = rowsum( (Sig12*Sig22m1):*(ey0, et) )
    CV = exp(lnsd1)^2 - Sig12*Sig22m1*Sig12'

    mc1   = m1 + CM
    y1Hat = mc1 + rnormal(rows(y),1,0,1)*sqrt(CV)
    y1    = tr:*y + (1 :-tr):*y1Hat

    mb1 = XX*X'(y1 - CM)
    vb1 = exp(lnsd1)^2*XX
    b1 = mb1 + cholesky(vb1)*rnormal(cols(b1), 1, 0, 1)
    b1 = b1'

    m1 = X*b1'
    ey1 = (y1 - m1)

Mata output:

: 
:     Sig12 = (v10, v1t)

:     Sig22 = exp(lnsd0)^2, v0t \ v0t, 1

:     Sig22m1 = invsym(Sig22)

:     CV = exp(lnsd1)^2 - Sig12*Sig22m1*Sig12'

:     y1Hat = mc1 + rnormal(rows(y),1,0,1)*sqrt(CV)

:     y1    = tr:*y + (1 :-tr):*y1Hat

:     vb1 = exp(lnsd1)^2*XX

:     b1 = mb1 + cholesky(vb1)*rnormal(cols(b1), 1, 0, 1)

:     b1 = b1'

:     ey1 = (y1 - m1)


# Typical MCMC - mwg draw

In [24]:
%%stata --mata
    prosd1 = 1
    gam = 1
    asta = .4

    lnsd1Hat = lnsd1 + rnormal(1,1,0,1)*prosd1
    Sigma    = exp(lnsd1)^2,   v10, v1t \ v10, exp(lnsd0)^2, v0t \ v1t, v0t, 1
    SigmaHat = exp(lnsd1Hat)^2,v10, v1t \ v10, exp(lnsd0)^2, v0t \ v1t, v0t, 1 

    if ( hasmissing(cholesky(SigmaHat)) == 0 ) {
        val    = ln_L((ey1, ey0, et), Sigma)
        valHat = ln_L((ey1, ey0, et), SigmaHat)
        rat = valHat - val
        alpha = min((exp(rat), 1))
        if (runiform(1,1,0,1) < alpha) lnsd1 = lnsd1Hat
            prosd1 = exp(gam*(alpha - asta))*prosd1
    }
    else {
        prosd1 = exp(-asta*gam)*prosd1
    }

Mata output:

:     prosd1 = 1

:     gam = 1

:     asta = .4

:     Sigma    = exp(lnsd1)^2,   v10, v1t \ v10, exp(lnsd0)^2, v0t \ v1t, v0t, 1

:     SigmaHat = exp(lnsd1Hat)^2,v10, v1t \ v10, exp(lnsd0)^2, v0t \ v1t, v0t, 1 
>         valHat = ln_L((ey1, ey0, et), SigmaHat)
>         alpha = min((exp(rat), 1))
>             prosd1 = exp(gam*(alpha - asta))*prosd1
>     else {
>     }
